In [21]:
#!pip install tensorwatch 
import azureml.core

print(azureml.core.VERSION, ' is the version of azureml sdk core')
import azureml.explain
print('--continue--')

import torch as T
print('--torch continue--')
print(T.__version__, ' torch version')

import numpy as np


from interpret.ext.blackbox import TabularExplainer
print('--interpret ok--')

#from azureml.contrib.interpret.visualize import ExplanationDashboard
#print('--contrib ok--')


1.0.72  is the version of azureml sdk core
--continue--
--torch continue--
1.2.0  torch version
--interpret ok--


ModuleNotFoundError: No module named 'azureml.contrib.interpret'

In [2]:
def accuracy(model, data_x, data_y, pct_close):
  n_items = len(data_y)
  X = T.Tensor(data_x)  # 2-d Tensor
  Y = T.Tensor(data_y)  # actual as 1-d Tensor

  oupt = model(X)       # all predicted as 2-d Tensor
  pred = oupt.view(n_items)  # all predicted as 1-d

  n_correct = T.sum((T.abs(pred - Y) < T.abs(pct_close * Y)))
  result = (n_correct.item() * 100.0 / n_items)  # scalar
  return result  

In [3]:
class Net(T.nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.hid1 = T.nn.Linear(13, 10)  # 13-(10-10)-1
    self.hid2 = T.nn.Linear(10, 10)
    self.oupt = T.nn.Linear(10, 1)

    T.nn.init.xavier_uniform_(self.hid1.weight)  # glorot
    T.nn.init.zeros_(self.hid1.bias)
    T.nn.init.xavier_uniform_(self.hid2.weight)
    T.nn.init.zeros_(self.hid2.bias)
    T.nn.init.xavier_uniform_(self.oupt.weight)
    T.nn.init.zeros_(self.oupt.bias)

  def forward(self, x):
    z = T.tanh(self.hid1(x))  # or T.nn.Tanh()
    z = T.tanh(self.hid2(z))
    z = self.oupt(z)  # no activation, aka Identity()
    return z


In [18]:

def main():
  # 0. get started
  print("\nBoston regression using PyTorch 1.0 \n")
  T.manual_seed(1);  np.random.seed(1)

  # 1. load data
  print("Loading Boston data into memory ")
  train_file = "boston_train.txt"
  test_file = "boston_test.txt"

  train_x = np.loadtxt(train_file, delimiter="\t",
    usecols=range(0,13), dtype=np.float32)
  train_y = np.loadtxt(train_file, delimiter="\t",
    usecols=[13], dtype=np.float32)
  test_x = np.loadtxt(test_file, delimiter="\t",
    usecols=range(0,13), dtype=np.float32)
  test_y = np.loadtxt(test_file, delimiter="\t",
    usecols=[13], dtype=np.float32)

  #look at our training data
  print((train_x))

  # 2. create model
  print("Creating 13-(10-10)-1 DNN regression model \n")
  net = Net()  # all work done above

  # 3. train model
  net = net.train()  # set training mode
  bat_size = 30 #was 10
  loss_func = T.nn.MSELoss()  # mean squared error
  optimizer = T.optim.SGD(net.parameters(), lr=0.01)
  n_items = len(train_x)
  batches_per_epoch = n_items // bat_size
  max_batches = 1000 * batches_per_epoch

  print("Starting training")
  for b in range(max_batches):
    curr_bat = np.random.choice(n_items, bat_size,
      replace=False)
    X = T.Tensor(train_x[curr_bat])
    Y = T.Tensor(train_y[curr_bat]).view(bat_size,1)
    optimizer.zero_grad()
    oupt = net(X)
      
    loss_obj = loss_func(oupt, Y)
    loss_obj.backward()  # compute gradients
    optimizer.step()     # update weights and biases

    if b % (max_batches // 10) == 0:
      print("batch = %6d" % b, end="")
      print("  batch loss = %7.4f" % loss_obj.item(), end="")
      net = net.eval()
      acc = accuracy(net, train_x, train_y, 0.15) 
      net = net.train()
      print("  accuracy = %0.2f%%" % acc)       
  print("Training complete \n")

  # 4. evaluate model
  net = net.eval()  # set eval mode
  acc = accuracy(net, test_x, test_y, 0.15) 
  print("Accuracy on test data = %0.2f%%" % acc)
  raw_inpt = np.array([[0.09266, 34, 6.09, 0, 0.433, 6.495, 18.4,
    5.4917, 7, 329, 16.1, 383.61, 8.67]], dtype=np.float32)

  norm_inpt = np.array([[0.000970, 0.340000, 0.198148, -1,
    0.098765, 0.562177, 0.159629, 0.396666, 0.260870, 0.270992,
    0.372340, 0.966488, 0.191501]], dtype=np.float32)

  y = net(X)
  X = T.Tensor(norm_inpt)
  
  print(repr(y.item))
  print(y[0])
  print("For a town with raw input values: ")
  for (idx,val) in enumerate(raw_inpt[0]):
    if idx % 5 == 0: print("")
    print("%11.6f " % val, end="")
    
    print("\n\nPredicted median house price = $%0.2f" % (y[idx]*10000))
  

  explainer = TabularExplainer(net, train_x) #, 
                             #features = boston_data.feature_names)
  global_explanation = explainer.explain_global(test_x)
  # Sorted SHAP values 
  print('ranked global importance values: {}'.format(global_explanation.get_ranked_global_values()))
  # Corresponding feature names
  print('ranked global importance names: {}'.format(global_explanation.get_ranked_global_names()))
  # Feature ranks (based on original order of features)
  print('global importance rank: {}'.format(global_explanation.global_importance_rank))

In [19]:
#net=Net()
main()
  


Boston regression using PyTorch 1.0 

Loading Boston data into memory 
[[4.83000e-04 3.30000e-01 5.33330e-02 ... 6.17021e-01 1.00000e+00
  1.60044e-01]
 [2.15000e-04 5.50000e-01 1.12593e-01 ... 5.31915e-01 1.00000e+00
  1.50386e-01]
 [2.50600e-03 0.00000e+00 2.28519e-01 ... 5.63830e-01 9.89510e-01
  4.71026e-01]
 ...
 [1.05400e-03 0.00000e+00 6.37040e-02 ... 5.53191e-01 1.00000e+00
  1.08996e-01]
 [1.67470e-02 0.00000e+00 6.97778e-01 ... 2.23404e-01 8.60558e-01
  3.18709e-01]
 [8.70620e-02 0.00000e+00 6.42963e-01 ... 8.08511e-01 6.85587e-01
  4.00110e-01]]
Creating 13-(10-10)-1 DNN regression model 

Starting training
batch =      0  batch loss =  4.7272  accuracy = 2.97%
batch =   1300  batch loss =  0.0898  accuracy = 69.06%
batch =   2600  batch loss =  0.1110  accuracy = 71.53%
batch =   3900  batch loss =  0.0694  accuracy = 72.28%
batch =   5200  batch loss =  0.1737  accuracy = 72.77%
batch =   6500  batch loss =  0.0918  accuracy = 73.51%
batch =   7800  batch loss =  0.1333  

In [20]:
from azureml.contrib.interpret.visualize import ExplanationDashboard

ModuleNotFoundError: No module named 'azureml.contrib.interpret'

In [ ]:
ExplanationDashboard(global_explanation, net, test_x)